In [1]:
#### Double commented means it was commented out but was on the cluster initially.

import sys,os,os.path
#sys.path.append("../../")   # cite IC from parent directory
##sys.path.append("/gluster/data/next/software/IC_satkill/")
#sys.path.append("/gluster/data/next/software/IC_sophronia/")
#sys.path.append(os.path.expanduser('~/code/eol_hsrl_python'))
##sys.path.append(os.path.expanduser('~/code/eol_hsrl_python'))
##os.environ['ICTDIR']='/gluster/data/next/software/IC_satkill/'
sys.path.append("../../")   # cite IC from parent directory
sys.path.append("/home/e78368jw/Documents/NEXT_CODE/next_misc/")
os.environ['ICTDIR']='/home/e78368jw/Documents/NEXT_CODE/IC'

#%load_ext autoreload
#%autoreload 2
#%matplotlib notebook

import time

import matplotlib.pyplot as plt
from matplotlib import rcParams
rcParams['mathtext.fontset'] = 'stix'
rcParams['font.family'] = 'STIXGeneral'
rcParams['figure.figsize'] = [10, 8]
rcParams['font.size'] = 22

import pandas as pd
import numpy  as np
import tables as tb

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.colors as clrs
from matplotlib.lines import Line2D


import IC.invisible_cities.core.core_functions                   as     coref
import IC.invisible_cities.io.dst_io                           as     dstio

from IC.invisible_cities.cities                 import beersheba as beerfun

from IC.invisible_cities.evm.event_model                          import HitCollection

from IC.invisible_cities.database.load_db       import DataSiPM

from IC.invisible_cities.evm.event_model        import Cluster, Hit
from IC.invisible_cities.types.ic_types         import xy
from IC.invisible_cities.reco.paolina_functions import voxelize_hits, drop_end_point_voxels, make_track_graphs, get_track_energy

from IC.invisible_cities.evm.event_model        import HitEnergy
from IC.invisible_cities.cities.beersheba          import DeconvolutionMode
from IC.invisible_cities.cities.beersheba          import CutType

from IC.invisible_cities.reco import hits_functions as hif

from IC.invisible_cities.reco.deconv_functions import deconvolve
from IC.invisible_cities.reco.deconv_functions import deconvolution_input
from IC.invisible_cities.reco.deconv_functions import InterpolationMethod

import IC.invisible_cities.io.mcinfo_io as mcio

from collections import defaultdict

import matplotlib.cm as cm
from matplotlib.colors import Normalize


In [4]:
file_no = 30
file = 'sophronia_' + str(file_no) + '_208Tl.h5'
##file = f'/gluster/data/next/files/TOPOLOGY_John/HYPPOS_DATA_QTHR/Q_THR4/PORT_1a/sophronia/sophronia_' + str(file_no) + '_208Tl.h5'

In [5]:
MC_df = pd.read_hdf(file, 'MC/particles')

In [6]:
#display(MC_df)
evt_id = MC_df[MC_df.particle_name == 'e+'].event_id.to_numpy()

In [7]:
print(evt_id)

[290013 290031 290032 290038 290044 290048 290051 290055 290058 290073
 290078 290081 290083 290099 290106 290108 290110 290112 290114 290125
 290140 290155 290159 290166 290173 290178 290215 290216 290218 290230
 290253 290261 290273 290278 290292 290304 290310 290315 290337 290342
 290353]


In [8]:
pd.set_option('display.max_rows', 500)
display(MC_df[MC_df.event_id == evt_id[0]])

,event_id,particle_id,particle_name,primary,mother_id,initial_x,initial_y,initial_z,initial_t,final_x,...,initial_momentum_x,initial_momentum_y,initial_momentum_z,final_momentum_x,final_momentum_y,final_momentum_z,kin_energy,length,creator_proc,final_proc
1044,290013,1,Tl208,True,0,396.107330,393.276245,534.039307,0.000000,396.107330,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,none,RadioactiveDecay
1045,290013,4,e-,False,1,396.107330,393.276245,534.039307,0.000183,404.240479,...,0.779324,1.095760,-0.359691,-0.000000,0.000000,0.000000,0.971746,1.545165e+01,RadioactiveDecay,eIoni
1046,290013,3,anti_nu_e,False,1,396.107330,393.276245,534.039307,0.000183,-8222.476562,...,-0.316265,0.444264,0.085812,-0.316265,0.444264,0.085812,0.552049,1.504398e+04,RadioactiveDecay,Transportation
1047,290013,2,Pb208[3475.078],False,1,396.107330,393.276245,534.039307,0.000183,396.107330,...,-0.463059,-1.540024,0.273879,-0.000000,-0.000000,0.000000,0.000007,0.000000e+00,RadioactiveDecay,RadioactiveDecay
1048,290013,6,gamma,False,2,396.107330,393.276245,534.039307,0.000183,460.731201,...,-0.527390,-0.391711,0.555856,0.000000,0.000000,-0.000000,0.860554,2.767462e+02,RadioactiveDecay,phot
1049,290013,21,e-,False,6,460.731201,331.111450,540.388916,0.923309,460.731201,...,-0.005523,-0.005359,0.000156,-0.000000,-0.000000,0.000000,0.000058,1.447787e-07,phot,msc
1050,290013,20,e-,False,6,460.731201,331.111450,540.388916,0.923309,460.731201,...,0.006145,-0.002717,-0.004100,-0.000000,0.000000,0.000000,0.000061,1.473075e-07,phot,msc
1051,290013,19,e-,False,6,460.731201,331.111450,540.388916,0.923309,460.731201,...,0.008027,0.014151,-0.023069,0.000000,0.000000,-0.000000,0.000779,1.385419e-05,phot,eIoni
1052,290013,18,gamma,False,6,460.731201,331.111450,540.388916,0.923309,460.740753,...,0.006884,0.002067,0.003525,0.000000,-0.000000,0.000000,0.008006,1.171419e-02,phot,phot
1053,290013,24,e-,False,18,460.740753,331.106476,540.393433,0.923348,460.740753,...,-0.022438,0.020111,-0.005542,-0.000000,0.000000,-0.000000,0.000918,1.704574e-05,phot,eIoni


### Need a bit of code to check and make sure that the parent particle of these positrons is Xe136

In [9]:
# select all positrons

# create tuple of event_id - mother_id to select from

positron_event_mothers = ((MC_df[MC_df.particle_name == 'e+']).event_id.to_numpy(), (MC_df[MC_df.particle_name == 'e+']).mother_id.to_numpy())
display(positron_event_mothers[0][10])
display(positron_event_mothers[1][10])
display(MC_df[MC_df.event_id == 290078])
display(MC_df[MC_df.particle_name == 'Xe131'])

290078

44

,event_id,particle_id,particle_name,primary,mother_id,initial_x,initial_y,initial_z,initial_t,final_x,...,initial_momentum_x,initial_momentum_y,initial_momentum_z,final_momentum_x,final_momentum_y,final_momentum_z,kin_energy,length,creator_proc,final_proc
6379,290078,1,Tl208,True,0,397.517181,393.651123,533.796814,0.000000,397.517181,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,none,RadioactiveDecay
6380,290078,4,e-,False,1,397.517181,393.651123,533.796814,0.000370,396.172668,...,0.226810,-0.541064,-0.082542,0.000000,0.000000,-0.000000,2.713859e-01,3.407538e+00,RadioactiveDecay,eIoni
6381,290078,3,anti_nu_e,False,1,397.517181,393.651123,533.796814,0.000370,-906.634766,...,-0.108366,-0.753754,0.994305,-0.108366,-0.753754,0.994305,1.252410e+00,1.507243e+04,RadioactiveDecay,Transportation
6382,290078,2,Pb208[3475.078],False,1,397.517181,393.651123,533.796814,0.000370,397.517181,...,-0.118445,1.294818,-0.911763,-0.000000,0.000000,-0.000000,6.510090e-06,0.000000e+00,RadioactiveDecay,RadioactiveDecay
6383,290078,6,gamma,False,2,397.517181,393.651123,533.796814,0.000370,414.921906,...,0.272301,-0.010746,-0.051663,0.000000,-0.000000,-0.000000,2.773668e-01,1.772853e+01,RadioactiveDecay,phot
6384,290078,12,e-,False,6,414.921906,392.964264,530.494629,0.059506,414.921906,...,0.002271,-0.007032,0.002156,0.000000,-0.000000,-0.000000,5.797000e-05,1.447787e-07,phot,msc
6385,290078,11,e-,False,6,414.921906,392.964264,530.494629,0.059506,414.921906,...,-0.004965,-0.005903,0.000208,-0.000000,-0.000000,0.000000,5.826000e-05,1.450558e-07,phot,msc
6386,290078,10,e-,False,6,414.921906,392.964264,530.494629,0.059506,414.921906,...,-0.002859,0.000033,-0.005115,-0.000000,0.000000,-0.000000,3.360000e-05,1.221262e-07,phot,msc
6387,290078,9,e-,False,6,414.921906,392.964264,530.494629,0.059506,414.921906,...,0.018118,-0.018195,-0.009791,0.000000,-0.000000,-0.000000,7.383700e-04,1.259782e-05,phot,eIoni
6388,290078,8,gamma,False,6,414.921906,392.964264,530.494629,0.059506,414.936340,...,0.001473,0.002954,0.007294,0.000000,0.000000,0.000000,8.005710e-03,7.840825e-02,phot,phot


,event_id,particle_id,particle_name,primary,mother_id,initial_x,initial_y,initial_z,initial_t,final_x,...,initial_momentum_x,initial_momentum_y,initial_momentum_z,final_momentum_x,final_momentum_y,final_momentum_z,kin_energy,length,creator_proc,final_proc
3120,290038,28,Xe131,False,25,-329.410553,152.934982,779.154663,2.691185,-329.410675,...,-0.342947,-0.645131,-0.134031,-0.0,-0.0,-0.0,0.000002,0.000309,conv,NoProcess
14488,290178,66,Xe131,False,63,186.531647,109.520004,773.333679,1.423868,186.531677,...,0.052582,-0.006367,0.990931,0.0,-0.0,0.0,0.000004,0.000412,conv,NoProcess
17632,290218,43,Xe131,False,40,-86.615578,82.166649,588.105042,1.924311,-86.615837,...,-0.629226,0.011447,-0.349185,-0.0,0.0,-0.0,0.000002,0.000299,conv,NoProcess
22073,290273,32,Xe131,False,29,108.491875,381.571106,885.146973,1.517249,108.491722,...,-0.366685,-0.736688,0.442954,-0.0,-0.0,0.0,0.000004,0.000388,conv,NoProcess


In [62]:
MC_df.final_proc.unique()

array(['RadioactiveDecay', 'eIoni', 'phot', 'msc', 'Transportation',
       'NoProcess', 'conv', 'annihil', 'UserSpecialCut'], dtype=object)

In [63]:
MC_df.creator_proc.unique()

array(['none', 'RadioactiveDecay', 'eBrem', 'phot', 'compt', 'eIoni',
       'conv', 'annihil'], dtype=object)

In [10]:
MC_df[MC_df.creator_proc == 'conv']
# this line here has the sauce, it shows the events oyu care about

,event_id,particle_id,particle_name,primary,mother_id,initial_x,initial_y,initial_z,initial_t,final_x,...,initial_momentum_x,initial_momentum_y,initial_momentum_z,final_momentum_x,final_momentum_y,final_momentum_z,kin_energy,length,creator_proc,final_proc
1085,290013,45,Cu65,False,42,402.410797,398.662048,547.204712,0.052081,402.410797,...,0.992594,-0.040111,0.144458,0.0,-0.0,0.0,8.333067e-06,1.445683e-06,conv,NoProcess
1086,290013,44,e+,False,42,402.410797,398.662048,547.204712,0.052081,402.561707,...,0.319938,0.809486,1.214907,0.0,-0.0,-0.0,1.068478e+00,9.174533e-01,conv,annihil
1128,290013,43,e-,False,42,402.410797,398.662048,547.204712,0.052081,402.434662,...,-0.253277,0.135666,0.852989,0.0,-0.0,0.0,5.240200e-01,3.302958e-01,conv,eIoni
2322,290031,40,Pb207,False,33,-839.102844,-34.995232,1319.603027,5.131673,-839.102844,...,-0.906009,0.287164,0.020794,-0.0,0.0,0.0,2.344284e-06,7.920021e-07,conv,NoProcess
2323,290031,39,e+,False,33,-839.102844,-34.995232,1319.603027,5.131673,-839.201111,...,-0.957210,-0.302034,0.610646,0.0,-0.0,-0.0,7.702054e-01,4.256005e-01,conv,annihil
2433,290031,38,e-,False,33,-839.102844,-34.995232,1319.603027,5.131673,-839.083252,...,0.068201,-0.577608,0.373589,-0.0,0.0,-0.0,3.486351e-01,1.569769e-01,conv,eIoni
2485,290032,50,Cu65,False,47,399.876465,387.161316,525.278625,0.052235,399.876465,...,0.106625,-0.306335,0.001459,0.0,-0.0,0.0,8.700226e-07,4.671279e-07,conv,NoProcess
2486,290032,49,e+,False,47,399.876465,387.161316,525.278625,0.052235,399.838470,...,0.425265,-1.011116,-1.014296,-0.0,0.0,-0.0,1.067963e+00,9.174178e-01,conv,annihil
2523,290032,48,e-,False,47,399.876465,387.161316,525.278625,0.052235,399.907715,...,0.352634,-0.785583,-0.264073,-0.0,-0.0,0.0,5.245423e-01,3.154818e-01,conv,eIoni
3120,290038,28,Xe131,False,25,-329.410553,152.934982,779.154663,2.691185,-329.410675,...,-0.342947,-0.645131,-0.134031,-0.0,-0.0,-0.0,2.263027e-06,3.086939e-04,conv,NoProcess


In [36]:
signal_df = pd.DataFrame(columns = MC_df.columns.values)

conv_slice = MC_df[MC_df.creator_proc == 'conv']
for e_id, df in conv_slice.groupby('event_id'):
    # check if conv selection has Xe isotope
    lst = df.particle_name.to_list()
    # truncate to only take 'Xe' as the input
    short_lst = []
    [short_lst.append(i[0:2]) for i in lst]
    if "Xe" in short_lst:
        display(df)
        print("Xe found in event {}".format(e_id))
        signal_df = pd.concat([signal_df, MC_df[MC_df.event_id == e_id]])

display(signal_df)


,event_id,particle_id,particle_name,primary,mother_id,initial_x,initial_y,initial_z,initial_t,final_x,...,initial_momentum_x,initial_momentum_y,initial_momentum_z,final_momentum_x,final_momentum_y,final_momentum_z,kin_energy,length,creator_proc,final_proc
3120,290038,28,Xe131,False,25,-329.410553,152.934982,779.154663,2.691185,-329.410675,...,-0.342947,-0.645131,-0.134031,-0.0,-0.0,-0.0,0.000002,0.000309,conv,NoProcess
3121,290038,27,e+,False,25,-329.410553,152.934982,779.154663,2.691185,-332.519196,...,-0.505731,0.022112,-0.069614,-0.0,-0.0,-0.0,0.211648,34.512207,conv,annihil
3157,290038,26,e-,False,25,-329.410553,152.934982,779.154663,2.691185,-245.353363,...,-1.512125,-0.167821,1.001654,0.0,-0.0,-0.0,1.380856,379.070221,conv,eIoni


Xe found in event 290038


,event_id,particle_id,particle_name,primary,mother_id,initial_x,initial_y,initial_z,initial_t,final_x,...,initial_momentum_x,initial_momentum_y,initial_momentum_z,final_momentum_x,final_momentum_y,final_momentum_z,kin_energy,length,creator_proc,final_proc
3618,290044,21,Xe129,False,18,-31.585411,362.857666,964.609985,2.032441,-31.585423,...,-0.027980,-0.129073,0.473258,-0.0,-0.0,0.0,0.000001,0.000204,conv,NoProcess
3619,290044,20,e+,False,18,-31.585411,362.857666,964.609985,2.032441,-29.040545,...,-0.357743,-0.177293,0.230968,0.0,0.0,0.0,0.177390,28.071030,conv,annihil
3665,290044,19,e-,False,18,-31.585411,362.857666,964.609985,2.032441,-16.297134,...,-1.444328,0.155733,1.156918,-0.0,-0.0,-0.0,1.415115,383.216431,conv,eIoni


Xe found in event 290044


,event_id,particle_id,particle_name,primary,mother_id,initial_x,initial_y,initial_z,initial_t,final_x,...,initial_momentum_x,initial_momentum_y,initial_momentum_z,final_momentum_x,final_momentum_y,final_momentum_z,kin_energy,length,creator_proc,final_proc
3901,290048,35,Xe129,False,32,294.954315,305.864624,308.7677,0.876499,294.954559,...,0.636000,-0.470344,-0.694365,0.0,-0.0,-0.0,0.000005,0.000437,conv,NoProcess
3902,290048,34,e+,False,32,294.954315,305.864624,308.7677,0.876499,258.061798,...,-0.597550,0.056914,-0.879747,0.0,-0.0,-0.0,0.670263,186.925034,conv,annihil
3937,290048,33,e-,False,32,294.954315,305.864624,308.7677,0.876499,213.382721,...,-1.054603,-0.504079,-0.653273,-0.0,0.0,-0.0,0.922239,288.516418,conv,eIoni


...

,event_id,particle_id,particle_name,primary,mother_id,initial_x,initial_y,initial_z,initial_t,final_x,...,initial_momentum_x,initial_momentum_y,initial_momentum_z,final_momentum_x,final_momentum_y,final_momentum_z,kin_energy,length,creator_proc,final_proc
26804,290337,35,Xe129,False,32,-188.727936,-232.105347,636.679871,2.884029,-188.728149,...,-0.531832,-0.255082,-0.175839,-0.0,-0.0,-0.0,0.000002,0.000256,conv,NoProcess
26805,290337,34,e+,False,32,-188.727936,-232.105347,636.679871,2.884029,-247.317276,...,-0.947371,-1.613477,0.156932,-0.0,0.0,-0.0,1.434911,424.788208,conv,annihil
26854,290337,33,e-,False,32,-188.727936,-232.105347,636.679871,2.884029,-192.305084,...,-0.289582,-0.033484,0.317674,-0.0,-0.0,0.0,0.157593,16.216393,conv,eIoni


Xe found in event 290337


,event_id,particle_id,particle_name,primary,mother_id,initial_x,initial_y,initial_z,initial_t,final_x,...,initial_momentum_x,initial_momentum_y,initial_momentum_z,final_momentum_x,final_momentum_y,final_momentum_z,kin_energy,length,creator_proc,final_proc
28207,290353,42,Xe129,False,39,202.613174,227.756912,632.450989,0.921263,202.613022,...,-0.341951,-0.435267,-0.012501,-0.0,-0.0,-0.0,0.000001,0.000230,conv,NoProcess
28208,290353,41,e+,False,39,202.613174,227.756912,632.450989,0.921263,202.296387,...,0.043773,-0.268706,0.184241,0.0,-0.0,0.0,0.096606,9.072005,conv,annihil
28266,290353,40,e-,False,39,202.613174,227.756912,632.450989,0.921263,183.060043,...,-1.532714,-0.906207,0.772075,0.0,0.0,0.0,1.495900,441.856415,conv,eIoni


Xe found in event 290353


,event_id,particle_id,particle_name,primary,mother_id,initial_x,initial_y,initial_z,initial_t,final_x,...,initial_momentum_x,initial_momentum_y,initial_momentum_z,final_momentum_x,final_momentum_y,final_momentum_z,kin_energy,length,creator_proc,final_proc
3096,290038,1,Tl208,True,0,399.094788,396.975037,532.901978,0.000000,399.094788,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,none,RadioactiveDecay
3097,290038,4,e-,False,1,399.094788,396.975037,532.901978,0.000011,399.597656,...,0.317499,-0.759870,-1.221578,-0.00000,-0.000000,0.000000,1.048354,2.959284,RadioactiveDecay,eIoni
3098,290038,3,anti_nu_e,False,1,399.094788,396.975037,532.901978,0.000011,-4433.631836,...,-0.149950,-0.200074,-0.404385,-0.14995,-0.200074,-0.404385,0.475439,15322.882812,RadioactiveDecay,Transportation
3099,290038,2,Pb208[3475.078],False,1,399.094788,396.975037,532.901978,0.000011,399.094788,...,-0.167549,0.959945,1.625964,-0.00000,0.000000,0.000000,0.000009,0.000000,RadioactiveDecay,RadioactiveDecay
3100,290038,6,gamma,False,2,399.094788,396.975037,532.901978,0.000011,433.833130,...,0.663099,0.249662,-0.488388,0.00000,-0.000000,-0.000000,0.860554,64.621887,RadioactiveDecay,phot
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28285,290353,108,e-,False,102,182.106796,150.249222,721.255676,1.398646,182.107101,...,0.012740,-0.000458,-0.003118,0.00000,-0.000000,-0.000000,0.000169,0.001002,phot,eIoni
28286,290353,107,e-,False,102,182.106796,150.249222,721.255676,1.398646,182.125565,...,0.092590,-0.026467,-0.076831,-0.00000,-0.000000,0.000000,0.014640,0.404148,phot,eIoni
28287,290353,101,e-,False,40,183.980576,147.294357,703.405212,1.336698,183.735596,...,-0.114430,-0.104642,-0.072460,0.00000,-0.000000,0.000000,0.027902,1.239423,eIoni,eIoni
28288,290353,100,e-,False,40,199.922455,176.166901,671.844849,1.157253,199.694321,...,-0.134355,-0.004156,0.091696,-0.00000,-0.000000,-0.000000,0.025281,1.114063,eIoni,eIoni


In [38]:
# save signal_df as events
signal_df.to_hdf('Tl_signal.h5', key = 'MC')

/home/e78368jw/anaconda3/envs/IC-3.8-2022-04-13/lib/python3.8/site-packages/pandas/core/generic.py:2703: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed-integer,key->block1_values] [items->Index(['event_id', 'particle_id', 'particle_name', 'primary', 'mother_id',
       'initial_volume', 'final_volume', 'creator_proc', 'final_proc'],
      dtype='object')]

  pytables.to_hdf(


In [40]:
load_test = pd.read_hdf('Tl_signal.h5', key = 'MC')

In [41]:
display(load_test)

,event_id,particle_id,particle_name,primary,mother_id,initial_x,initial_y,initial_z,initial_t,final_x,...,initial_momentum_x,initial_momentum_y,initial_momentum_z,final_momentum_x,final_momentum_y,final_momentum_z,kin_energy,length,creator_proc,final_proc
3096,290038,1,Tl208,True,0,399.094788,396.975037,532.901978,0.000000,399.094788,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,none,RadioactiveDecay
3097,290038,4,e-,False,1,399.094788,396.975037,532.901978,0.000011,399.597656,...,0.317499,-0.759870,-1.221578,-0.00000,-0.000000,0.000000,1.048354,2.959284,RadioactiveDecay,eIoni
3098,290038,3,anti_nu_e,False,1,399.094788,396.975037,532.901978,0.000011,-4433.631836,...,-0.149950,-0.200074,-0.404385,-0.14995,-0.200074,-0.404385,0.475439,15322.882812,RadioactiveDecay,Transportation
3099,290038,2,Pb208[3475.078],False,1,399.094788,396.975037,532.901978,0.000011,399.094788,...,-0.167549,0.959945,1.625964,-0.00000,0.000000,0.000000,0.000009,0.000000,RadioactiveDecay,RadioactiveDecay
3100,290038,6,gamma,False,2,399.094788,396.975037,532.901978,0.000011,433.833130,...,0.663099,0.249662,-0.488388,0.00000,-0.000000,-0.000000,0.860554,64.621887,RadioactiveDecay,phot
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28285,290353,108,e-,False,102,182.106796,150.249222,721.255676,1.398646,182.107101,...,0.012740,-0.000458,-0.003118,0.00000,-0.000000,-0.000000,0.000169,0.001002,phot,eIoni
28286,290353,107,e-,False,102,182.106796,150.249222,721.255676,1.398646,182.125565,...,0.092590,-0.026467,-0.076831,-0.00000,-0.000000,0.000000,0.014640,0.404148,phot,eIoni
28287,290353,101,e-,False,40,183.980576,147.294357,703.405212,1.336698,183.735596,...,-0.114430,-0.104642,-0.072460,0.00000,-0.000000,0.000000,0.027902,1.239423,eIoni,eIoni
28288,290353,100,e-,False,40,199.922455,176.166901,671.844849,1.157253,199.694321,...,-0.134355,-0.004156,0.091696,-0.00000,-0.000000,-0.000000,0.025281,1.114063,eIoni,eIoni


In [ ]:
# develop a function that can load in files, collect them like this, and add to a large dataframe.
def collect_signal_df(data_path, save_path = "", verbose = True):
    """
    Function that collects Tl208 signal events (identified with Xe ions created in 'conv' processes)
    """

     # collect all filenames
    try:
        file_names = [f for f in os.listdir(data_path) if os.path.isfile(os.path.join(data_path, f)) and f.endswith('.h5')]
    except:
        print("File path incorrect, please state the correct file path\n(but not any particular folder!)")

    # counter for creating first array
    i = 0


    for file in file_names:
        file_path = data_path + file

        # load in data
        MC_df = pd.read_hdf(file_path, 'MC/particles')
        
        # set first dataframe
        if (i == 0):
            MC_signal_df = pd.DataFrame(columns = MC_df.columns.values)
            i = 1
        
        # collect conv slice and iterate over, scanning for Xe*** events
        conv_slice = MC_df[MC_df.creator_proc == 'conv']
        for e_id, df in conv_slice.groupby('event_id'):
            # check if conv selection has Xe isotope
            lst = df.particle_name.to_list()
            # truncate to only take 'Xe' as the input
            short_lst = []
            [short_lst.append(i[0:2]) for i in lst]
            if "Xe" in short_lst:
                if (verbose == True):
                    display(df)
                    print("Xe found in event {}".format(e_id))
                MC_signal_df = pd.concat([MC_signal_df, MC_df[MC_df.event_id == e_id]])

    # save if desired
    if save_path != "":
        MC_signal_df.to_hdf('Tl_signal.h5', key = 'MC')

    # return dataframe    
    return MC_signal_df

